<a href="https://colab.research.google.com/github/cyin666/SDOH/blob/main/SDOH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SDOH Project**

##Terminologies:
1. CCU: Coronary Care Unit:


> Specialized in the care of patients with heart attacks, unstable angina, cardiac dysrhythmia and (in practice) various other cardiac conditions that require continuous monitoring and treatment.



2. CSRU: Cardiac Surgery Recovery Unit:

> Specialized intensive care unit for cardiac surgery patients immediately following surgery.



3. MICU: Medical Intensive Care Unit


> Provide care for patients with medical conditions who do not require surgery



4. SICU: Surgical Intensive Care Unit:



> A specialized service in larger hospitals that provides inpatient care for critically ill patients on surgical services. 

5. TSICU: Trauma Surgical Intensive Care Unit


> Found in hospitals certified in treating major trauma with a dedicated trauma team equipped with the expertise to deal with serious complications.

6. NICU: Neonatal intensive care unit


> Found in hospitals certified in treating major trauma with a dedicated trauma team equipped with the expertise to deal with serious complications.

7. NWARD: Neonatal ward


> Provides around-the-clock care to sick or premature babies. 



##Useful Links:
1. Official mimic data documentation: https://mimic.mit.edu/docs/iii/
2. Schema overview and description: https://mit-lcp.github.io/mimic-schema-spy/
3. Table relationships: https://mit-lcp.github.io/mimic-schema-spy/relationships.html

## Connect to Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Liabraries

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Import Datasets

In [10]:
#Module 1: The following tables are used to define and track patient stays:

#t1:Hospital admissions associated with an ICU stay.
#keys: HADM_ID, PATIENTS on SUBJECT_ID
admissions = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz',compression='gzip')

#t2:Record of when patients were ready for discharge (called out), and the actual time of their discharge (or more generally, their outcome).
#keys:PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID
callout = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/CALLOUT.csv.gz',compression='gzip')

#t3:List of ICU admissions.
#keys: ICUSTAY_ID, PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID
icustays = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/ICUSTAYS.csv.gz',compression='gzip')

#t4:Patients associated with an admission to the ICU.
#keys: SUBJECT_ID,ADMISSIONS on SUBJECT_ID,ICUSTAYS on SUBJECT_ID
patients = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/PATIENTS.csv.gz',compression='gzip')

#t5:Hospital services that patients were under during their hospital stay.
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID
services = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/SERVICES.csv.gz',compression='gzip')

#t6:	Location of patients during their hospital stay.
#keys:PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID
transfers = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/TRANSFERS.csv.gz',compression='gzip')

In [17]:
#Module 2: The following tables contain data collected in the critical care unit:

#t7: List of caregivers associated with an ICU stay.
#keys: CHARTEVENTS on CGID
caregivers = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/CAREGIVERS.csv.gz',compression='gzip')


#t8: Events occuring on a patient chart. ***330 M rows
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID,D_ITEMS on ITEMID,CAREGIVERS on CGID
chartevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz',compression='gzip')

#t9: Events relating to a datetime. ***4 M rows
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID,D_ITEMS on ITEMID,CAREGIVERS on CGID
datetimeevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/DATETIMEEVENTS.csv.gz',compression='gzip')

#t10: Events relating to fluid input for patients whose data was originally stored in the CareVue database. ***17M rows
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID,D_ITEMS on ITEMID,CAREGIVERS on CGID
inputevents_cv = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/INPUTEVENTS_CV.csv.gz',compression='gzip')

#t11: Events relating to fluid input for patients whose data was originally stored in the MetaVision database. ***3M rows
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID,D_ITEMS on ITEMID,CAREGIVERS on CGID
inputevents_mv = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/INPUTEVENTS_MV.csv.gz',compression='gzip')

#t12: Notes associated with hospital stays. #2M rows
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,CAREGIVERS on CGID
noteevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv.gz',compression='gzip')

#t13: Outputs recorded during the ICU stay.
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID,D_ITEMS on ITEMID,CAREGIVERS on CGID
outputevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/OUTPUTEVENTS.csv.gz',compression='gzip')

#t14: Procedure start and stop times recorded for MetaVision patients.
#keys:#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID,ICUSTAYS on ICUSTAY_ID,D_ITEMS on ITEMID
procedureevents_mv = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/PROCEDUREEVENTS_MV.csv.gz',compression='gzip')

In [32]:
#Module 3: The following tables contain data collected in the hospital record system:
#t15: Events recorded in Current Procedural Terminology.
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID
cptevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/CPTEVENTS.csv.gz',compression='gzip')

#t16: Diagnoses relating to a hospital admission coded using the ICD9 system.
#keys: PATIENTS on SUBJECT_ID,ADMISSIONS on HADM_ID.D_ICD_DIAGNOSES on ICD9_CODE
diagnoses_icd = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz',compression='gzip')

#t17: Hospital stays classified using the Diagnosis-Related Group system.
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID
drgcodes = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/DRGCODES.csv.gz',compression='gzip')

#t18: Events relating to laboratory tests.
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID, D_LABITEMS on ITEMID
labevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/LABEVENTS.csv.gz',compression='gzip')

#t19: Events relating to microbiology tests.
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID, D_ITEMS on SPEC_ITEMID, D_ITEMS on ORG_ITEMID, D_ITEMS on AB_ITEMID
microbiologyevents = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/MICROBIOLOGYEVENTS.csv.gz',compression='gzip')

#t20: Medicines prescribed.
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID, ICUSTAYS on ICUSTAY_ID
prescriptions = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz',compression='gzip')

#t21: Procedures relating to a hospital admission coded using the ICD9 system.
#keys: PATIENTS on SUBJECT_ID, ADMISSIONS on HADM_ID, D_ICD_PROCEDURES on ICD9_CODE
procedures_icd = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/PROCEDURES_ICD.csv.gz',compression='gzip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
#Module 4: The following tables are dictionaries:
#t22: High-level dictionary of the Current Procedural Terminology.
#keys: CPTEVENTS on CPT_CD between MINCODEINSUBSECTION and MAXCODEINSUBSECTION
d_cpt = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/D_CPT.csv.gz',compression='gzip')

#t23: Dictionary of the International Classification of Diseases, 9th Revision (Diagnoses).
#keys: DIAGNOSES_ICD ON ICD9_CODE
d_icd_diagnoses = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz',compression='gzip')

#t24: Dictionary of the International Classification of Diseases, 9th Revision (Procedures).
#keys:PROCEDURES_ICD on ICD9_CODE
d_icd_procedures = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/D_ICD_PROCEDURES.csv.gz',compression='gzip')

#t25: Dictionary of non-laboratory-related charted items.
#*** All Metavision ITEMIDs will have a value > 220000
#keys:
#CHARTEVENTS on ITEMID
#DATETIMEEVENTS on ITEMID
#INPUTEVENTS_CV on ITEMID
#INPUTEVENTS_MV on ITEMID
#MICROBIOLOGYEVENTS on SPEC_ITEMID, ORG_ITEMID, or AB_ITEMID (for example, use d_items.ITEMID = microbiologyevents.SPEC_ITEMID)
#OUTPUTEVENTS on ITEMID
#PROCEDUREEVENTS_MV on ITEMID
d_items = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/D_ITEMS.csv.gz',compression='gzip')

#t26: Dictionary of laboratory-related items.
#keys: LABEVENTS on ITEMID
d_labitems = pd.read_csv('/content/drive/MyDrive/SDOH/mimic-iii-clinical-database-1.4/D_LABITEMS.csv.gz',compression='gzip')

## Exloratory Data Analysis

In [37]:
d_icd_diagnoses.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
